# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

## Notes
Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

# Q1

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import re

In [3]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [4]:
teams = cities['NHL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')               
teams['Metropolitan area'] = cities['Metropolitan area']
teams = pd.melt(teams, id_vars=['Metropolitan area']).replace('',np.nan).dropna().rename(columns={'value':'team'}).reset_index().drop(columns=['variable','index'])

In [5]:
teams = teams.merge(cities,how = 'left',on = 'Metropolitan area').iloc[:,0:3].rename(columns={'Population (2016 est.)[8]':'Population'})
teams = teams.astype({'Metropolitan area':str,'team':str,'Population':int})
teams['team']=teams['team'].str.replace('[\w.]*\ ','')

In [6]:
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year']==2018]
nhl_df = nhl_df[nhl_df['GP'].apply(lambda x:x.isnumeric())]
nhl_df = nhl_df[['team','W','L']]
nhl_df['team'] = nhl_df['team'].apply(lambda x: x.split(' ')[-1].strip('*'))
nhl_df = nhl_df.astype({'W':int,'L':int})
nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])

In [7]:
merge = pd.merge(teams,nhl_df,how = 'outer', on = 'team')

In [8]:
merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})

In [9]:
stats.pearsonr(merge['Population'], merge['W/L%'])

(0.0124861629212099, 0.9497182859911673)

## Q2 
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.

In [114]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
teams = cities['NBA'].str.extract('([A-Z0-9]{0,2}\w*\ [A-Z]{0,2}\w*|[A-Z0-9]{0,2}[a-z]*)([A-Z0-9]{0,2}\w*|[A-Z0-9]{0,2}\s\w*)')
teams['Metropolitan area'] = cities['Metropolitan area']
teams = pd.melt(teams,id_vars='Metropolitan area').reset_index().drop(columns=['variable','index']).replace('',np.nan).dropna().rename(columns={'value':'team'})
teams = teams.merge(cities,how = 'left', on = 'Metropolitan area').iloc[:,[0,1,2]].rename(columns={'Population (2016 est.)[8]':'Population'}).astype({'Population':int})
teams['team'] = teams['team'].apply(lambda x: x.split(' ')[-1])

In [116]:
nba_df = pd.read_csv("assets/nba.csv")
nba_df = nba_df[nba_df['year']==2018]
nba_df = nba_df.iloc[:,[0,1,2]]
nba_df['team'] = nba_df['team'].apply(lambda x: x.split('(')[0])
nba_df['team'] = nba_df['team'].apply(lambda x: x.split('*')[0])
nba_df['team'] = nba_df['team'].apply(lambda x: x.split(' ')[-1].strip())
nba_df = nba_df.astype({'W':int,'L':int})
nba_df['W/L%'] = nba_df['W']/(nba_df['W']+nba_df['L'])

In [118]:
merge = pd.merge(nba_df,teams,how = 'left',on = 'team')
merge = merge.groupby('Metropolitan area').agg({'W/L%':np.mean,'Population':np.mean})
merge['Population'].shape


(28,)

In [119]:
stats.pearsonr(merge['Population'], merge['W/L%'])

(-0.17657160252844617, 0.3687474160446289)

In [120]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [ ]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]